In [1]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

%matplotlib inline

In [2]:
airlines = pd.read_csv('flight-delays/airlines.csv')
airlines.shape

(14, 2)

In [3]:
airlines.head(1)

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.


In [4]:
flights = pd.read_csv('flight-delays/clean_flights.csv')
flights.shape

(5819079, 31)

In [5]:
flights.head(1)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
airports = pd.read_csv('flight-delays/airports.csv')
airports.shape

(322, 7)

In [7]:
airports.head(1)

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.4404


### Create column for date

In [8]:
# first creat method edit that fix numbers format, e.g: 1 -> 01

def edit_number(num):
    if len(num) == 1:
        return '0'+ str(num)
    else:
        return num

In [9]:
flights['MONTH'] = flights['MONTH'].apply(lambda x: edit_number(str(x)))
flights['DAY'] = flights['DAY'].apply(lambda x: edit_number(str(x)))

flights.sample(3)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
5504332,2015,10,11,7,B6,1654,N635JB,SJU,FLL,2034,...,2302.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2823809,2015,06,26,5,OO,2845,N869AS,LAX,SJC,2155,...,47.0,101.0,0,0,NaN,1.0,0.0,0.0,100.0,0.0
5360480,2015,10,02,5,DL,456,N698DL,JFK,SLC,1600,...,2003.0,43.0,0,0,NaN,26.0,0.0,17.0,0.0,0.0


In [10]:
flights['date'] = flights[['YEAR','MONTH','DAY']].apply(lambda row: str(row['DAY'])\
                            +'/'+str(row['MONTH'])+'/'+str(row['YEAR']),axis=1)

In [11]:
flights.sample(3)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,date
4793278,2015,11,27,5,WN,5594,N647SW,ATL,PBI,955,...,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,27/11/2015
3969079,2015,09,03,4,B6,90,N562JB,SAN,JFK,2041,...,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,03/09/2015
116230,2015,01,08,4,MQ,3530,N537MQ,CVG,ORD,1136,...,160.0,0,0,NaN,111.0,0.0,0.0,49.0,0.0,08/01/2015


In [12]:
flights['date'].dtypes

dtype('O')

In [13]:
#  convert date column type from object to date 

flights['date'] = flights.apply(lambda row: datetime.strptime(row['date'], '%d/%m/%Y'),axis=1)

flights['date'].dtypes

dtype('<M8[ns]')

In [14]:
flights.sample(3)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,date
1030436,2015,03,09,1,EV,5121,N730EV,LGA,ROC,850,...,7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-09
2969712,2015,07,06,1,AA,248,N4YPAA,DFW,BNA,700,...,-1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-06
1176554,2015,03,18,3,WN,884,N764SW,PIT,PHX,715,...,-31.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-18


### Define a flag column for delay

In [15]:
# Define a flag column for delay (Departure or Arrival) : 15 Mints >=

flights['delayed'] = flights.apply(lambda x:1 if(x['ARRIVAL_DELAY']>=15 or x['DEPARTURE_DELAY']>=15) else 0,axis=1)


### Define a flag column Shift (Morning, Afternoon, Noon, Night)

In [16]:
# Combine flights per shift and apply it to data frame
def Day_Shift(x):
    if x in range(500,1200):
        return 'Morninig'
    elif x in range(1201,1700):
        return 'Afternoon'
    elif x in range(1701,2100):
        return 'Evening'
    else:
        return 'Night'
    
flights['FlightsShift'] = flights['SCHEDULED_DEPARTURE'].apply(Day_Shift)
flights.head(1)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,date,delayed,FlightsShift
0,2015,01,01,4,AS,98,N407AS,ANC,SEA,5,...,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,Night


### Define a flag column Seasons 

#### Seasons are:
1. Winter: December, January, and Fibruary.
2. Spring: March, April, and May.
3. Summer: June, July, and August.
4. Autumn: September, October, and November.

In [17]:
# Determine Season for each Flight

def Year_Shift(x):
    if ((x == '03') or (x == '04') or (x == '05')):
        return 'Spring'
    elif ((x == '06') or (x == '07') or (x == '08')):
        return 'Summer'
    elif ((x == '09') or (x == '10') or (x == '11')):
        return 'Autumn'
    elif ((x == '12') or (x == '01') or (x == '02')):
        return 'Winter'
    
flights['SEASON'] = flights['MONTH'].apply(Year_Shift)
flights.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,date,delayed,FlightsShift,SEASON
0,2015,01,01,4,AS,98,N407AS,ANC,SEA,5,...,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,Night,Winter
1,2015,01,01,4,AA,2336,N3KUAA,LAX,PBI,10,...,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,Night,Winter
2,2015,01,01,4,US,840,N171US,SFO,CLT,20,...,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,Night,Winter
3,2015,01,01,4,AA,258,N3HYAA,LAX,MIA,20,...,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,Night,Winter
4,2015,01,01,4,AS,135,N527AS,SEA,ANC,25,...,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,Night,Winter


### Full dataset

xxxxxxxxx

In [ ]:
airports = airports.drop('COUNTRY',axis=1)
airports.columns

In [29]:
# join the airports info with flights info

left = flights
right = airports
right.columns = ['ORIGIN_AIRPORT', 'ORIGIN_AIRPORT_NAME', 'ORIGIN_CITY',\
                 'ORIGIN_STATE', 'ORIGIN_LATITUDE', 'ORIGIN_LONGITUDE']
full_data = pd.merge(left, right, how='left', on='ORIGIN_AIRPORT')

full_data.shape
# 5819079

(5819079, 40)

In [30]:
# join the airports info with flights info

left = full_data
right = airports
right.columns = ['DESTINATION_AIRPORT', 'DESTINATION_AIRPORT_NAME', 'DESTINATION_CITY',\
                 'DESTINATION_STATE', 'DESTINATION_LATITUDE', 'DESTINATION_LONGITUDE']
full_data = pd.merge(left, right, how='left', on='DESTINATION_AIRPORT')

full_data.shape
# 5819079

(5819079, 45)

In [35]:
# join the airlines info with flights info

left = full_data
right = airlines
right.columns = ['AIRLINE','AIRLINE NAME']
full_data = pd.merge(left, right, how='left', on='AIRLINE')

full_data.shape
# 5819079

(5819079, 46)

In [37]:
# save the full_data in CSV file

url = '../ML-Project/flight-delays/full_data.csv'

full_data.to_csv(url,index=False)